# 応用演習2<パーセプトロン>


---
　この演習では、ニューラルネットワークの原点であるパーセプトロンの実験を行います。この演習を通して、層を深くすることと活性化関数の効果を体感できます。

---


>　パーセプトロンは、人間の脳機能をモデル化したもので、複数の信号を受けて一つの信号を出力することができます。入力信号*x*と重み*w*の積の総和を出力とします。以下に、シンプルなパーセプトロンの図を示します。




<img src="https://drive.google.com/uc?id=1WVr987sZV5xU3ul6iPjIiLFjzskFhqLD" alt="" title="neuron" width=30%>



> このパーセプトロンをPytorchを使って再現します。



>>(厳密には、単純パーセプトロンとは異なるモデルです。初期のパーセプトロンでは、出力に対してしきい値関数を用いて1または0を決定します。しかし、しきい値関数は微分可能でない点があるのでバックプロパゲーションと相性が良くありません。)









> 初めに、使用するモジュールのインポートします。



In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader


> 次の部分で、パーセプトンもどきを定義します。nn.Linear()で入力データ*x*に重みを掛け合わせます。続く、Sigmoid関数は最初期のパーセプトロンでは用いられていませんが、1.バックプロパゲーションと2.精度を安定させるために使用します。




In [ ]:
class SimpleNet(nn.Module):
    def __init__(self):
        super().__init__()
        #全結合層(入力2,出力1)
        self.fc = nn.Linear(2,1)
        #活性化関数(sigmoid関数)
        self.act = nn.Sigmoid()
    def forward(self,x):
        y = self.fc(x)
        y = self.act(y)
        return y


> 次に、入力データ*x*で用いるANDゲートの入力と出力を設定します。



In [ ]:
### AND gate ####
and_data = []
and_data.append(([0,0]))  # 0&0=0
and_data.append(([0,1]))  # 0&1=0
and_data.append(([1,0]))  # 1&0=0
and_data.append(([1,1]))  # 1&1=1

### 正解ラベル ###
and_target_label = [0,0,0,1]

for i in range(len(and_data)):
    print(f"Input {and_data[i]}, Output {and_target_label[i]}")

Input [0, 0], Output 0
Input [0, 1], Output 0
Input [1, 0], Output 0
Input [1, 1], Output 1




> 続いては、データセット(入力するデータを集約してネットワークに渡す)を作成します。自作のデータセットは、今後必須になってくるので後ほど理解できるようになると良いと思います。



In [ ]:
class MyDataset(Dataset):
    def __init__(self,data,label):
        self.data_info = list()
        for idx,data in enumerate(data):
            _data = torch.tensor(data).to(torch.float32)
            _label = torch.tensor(label)
            self.data_info.append((_data,_label[idx]))
    def __getitem__(self,index):
        data,label = self.data_info[index]
        return data, label
    def __len__(self):
        return len(self.data_info)

In [ ]:
dataset = MyDataset(and_data, and_target_label)
dataloader = DataLoader(dataset, batch_size=4)



> 以下は、学習要件の定義と訓練部分です。

専門用語 :
*   エポック
 *  エポックとは、ハイパーパラメータ(手動で設定する値)の一種で、訓練データ全体を何回繰り返して学習を行うかを決めます。
*   損失関数
 *  正解の値と、モデルによる予測結果の誤差を数値化します。
*   Optimizer
  * 械学習モデルは、試行錯誤を繰り返して重みを更新し、損失0を目指しますが、Optimzerは損失を効率よく0に近づける最適化アルゴリズムです。例をあげると、勾配降下法では、勾配(傾き)を求めて斜面を下る方向に重みを更新し、損失の最小値を求めます。
*   バックプロパゲーション
 * 勾配降下法では、勾配を求めるために、微分する必要があります。バックプロパゲーションでは、出力層から入力層に向かって誤差の偏微分を伝番していき、各層の微分を求めます。





In [ ]:
#エポック数
num_epochs = 50
#ネットワークの
net = SimpleNet()
# 損失関数の定義
criterion = nn.BCELoss()
optimizer = optim.SGD(net.parameters(), lr=0.01,
                      momentum=0.9, weight_decay=5e-4)

for epoch in range(num_epochs):
    correct = 0
    total_loss = 0
    total = 0
    for i,(x,label) in enumerate(dataloader):
        y_ = net(x)
        label_tensor = torch.unsqueeze(label,dim=1).to(torch.float32)
        loss = criterion(y_, label_tensor)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        # 出力に対して、しきい値を設定し、0か1の出力を得ます。
        pred = torch.heaviside((y_*2-1),torch.zeros_like(y_))
        correct += (pred.squeeze() == label).sum().item()
        total += x.shape[0]

    print('Epoch : {}\t'.format(epoch),
        'Loss : {:.3f}\t'.format(total_loss/(i+1)),
        'Acc : {}\t'.format((correct/total)*100))
    if ((epoch+1) % 50 == 0):
        for i in range(x.shape[0]):
            print('In : {}\t Out : {}\t Target : {}\n'.format(x[i].numpy(), pred[i].detach().numpy(),label[i]))




> 学習率(learning rate)、エポック数、シード値がかみ合うと、正解率100%が得られます。





> 次に、XORゲートの出力を予測します。



In [ ]:
xor_data = []
xor_data.append((0,0))
xor_data.append((0,1))
xor_data.append((1,0))
xor_data.append((1,1))
xor_target_label = [0,1,1,0]



> 先ほどと同様に、モデルの訓練をしてみましょう。



In [ ]:
dataset = MyDataset(xor_data, xor_target_label)
dataloader = DataLoader(dataset, batch_size=4)
#エポック数
num_epochs = 50
#ネットワークの
net = SimpleNet()
# 損失関数の定義
criterion = nn.BCELoss()
optimizer = optim.SGD(net.parameters(), lr=0.01,
                      momentum=0.9, weight_decay=5e-4)

for epoch in range(num_epochs):
    correct = 0
    total_loss = 0
    total = 0
    for i,(x,label) in enumerate(dataloader):
        y_ = net(x)
        label_tensor = torch.unsqueeze(label,dim=1).to(torch.float32)
        loss = criterion(y_, label_tensor)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        # 出力に対して、しきい値を設定し、0か1の出力を得ます。
        pred = torch.heaviside((y_*2-1),torch.zeros_like(y_))
        correct += (pred.squeeze() == label).sum().item()
        total += x.shape[0]

    if (epoch % 5==0):
        print('Epoch : {}\t'.format(epoch),
            'Loss : {:.3f}\t'.format(total_loss/(i+1)),
            'Acc : {}\t'.format((correct/total)*100))


for i in range(x.shape[0]):
    print('In : {}\t Out : {}\t Target : {}\n'.format(x[i].numpy(), pred[i].detach().numpy(),label[i]))





> 高い正解率を得るのが難しいですね。これは、ネットワークの根本的な問題が原因です。パーセプトロンは、出力を*y = w1x1 + w2x2*で表現しますが、これだと直線で分割できる問題しか解決できません。

<img src="https://drive.google.com/uc?id=11-jZyRpaVZN1npZbBlynfjs7VR-H1kqy" alt="" title="neuron" width=60%>


> これを解決する方法は、1. 層を増やす、2. 活性化関数(非線形関数)を加える ことです。これにより、非線形分離ができるようになり、XORの予測ができます。それでは、SimpleNet()を改良してみましょう。




In [ ]:
class MultiLayerNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(2,4)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(4,1)
        self.act = nn.Sigmoid()
    def forward(self,x):
        y = self.fc1(x)
        y = self.relu(y)
        y = self.fc2(y)
        y = self.act(y)
        return y



> データセットをXOR用に変更します。



In [ ]:
dataset = MyDataset(xor_data, xor_target_label)
dataloader = DataLoader(dataset, batch_size=4)

In [ ]:
#エポック数
num_epochs = 50
#ネットワークの
net = MultiLayerNet()
# 損失関数の定義
criterion = nn.BCELoss()
optimizer = optim.SGD(net.parameters(), lr=0.01,
                      momentum=0.9, weight_decay=5e-4)

for epoch in range(num_epochs):
    correct = 0
    total_loss = 0
    total = 0
    for i,(x,label) in enumerate(dataloader):
        y_ = net(x)
        label_tensor = torch.unsqueeze(label,dim=1).to(torch.float32)
        loss = criterion(y_, label_tensor)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        # 出力に対して、しきい値を設定し、0か1の出力を得ます。
        pred = torch.heaviside((y_*2-1),torch.zeros_like(y_))
        correct += (pred.squeeze() == label).sum().item()
        total += x.shape[0]
    if ((epoch+1)%5==0):
        print('Epoch : {}\t'.format(epoch),
            'Loss : {:.3f}\t'.format(total_loss/(i+1)),
            'Acc : {}\t'.format((correct/total)*100))

for i in range(x.shape[0]):
    print('In : {}\t Out : {}\t Target : {}\n'.format(x[i].numpy(), pred[i].detach().numpy(),label[i]))

Epoch : 4	 Loss : 0.681	 Acc : 50.0	
Epoch : 9	 Loss : 0.679	 Acc : 50.0	
Epoch : 14	 Loss : 0.677	 Acc : 75.0	
Epoch : 19	 Loss : 0.674	 Acc : 75.0	
Epoch : 24	 Loss : 0.671	 Acc : 75.0	
Epoch : 29	 Loss : 0.669	 Acc : 100.0	
Epoch : 34	 Loss : 0.666	 Acc : 100.0	
Epoch : 39	 Loss : 0.663	 Acc : 100.0	
Epoch : 44	 Loss : 0.659	 Acc : 100.0	
Epoch : 49	 Loss : 0.656	 Acc : 100.0	
In : [0. 0.]	 Out : [0.]	 Target : 0

In : [0. 1.]	 Out : [1.]	 Target : 1

In : [1. 0.]	 Out : [1.]	 Target : 1

In : [1. 1.]	 Out : [0.]	 Target : 0





> ネットワークや、学習率、エポック数を変更して、正解率100%を目指しましょう。

